In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 2.0MB 17.8MB/s 
     |████████████████████████████████| 1.2MB 44.5MB/s 
     |████████████████████████████████| 890kB 51.9MB/s 
     |████████████████████████████████| 3.2MB 50.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.2-cp37-none-any.whl size=114270 sha256=5838b99acfa3831e596cc2e785bd4a050a8b65c3c7aa4a798bbfea071d95e8f7
  Stored in directory: /root/.cache/pip/wheels/c7/45/c2/1e55040d016e678dd9378d96b86b7632a4d820f66c60566c05
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=ff99f29d73a7a4b737e983a318dad140959f0e09ea397ea5ec3b926b26d8dc59
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Mar 21 16:58:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
"""
This examples show how to train a Bi-Encoder for the MS Marco dataset (https://github.com/microsoft/MSMARCO-Passage-Ranking).
The queries and passages are passed independently to the transformer network to produce fixed sized embeddings.
These embeddings can then be compared using cosine-similarity to find matching passages for a given query.
For training, we use MultipleNegativesRankingLoss. There, we pass triplets in the format:
(query, positive_passage, negative_passage)
Negative passage are hard negative examples, that where retrieved by lexical search. We use the negative
passages (the triplets) that are provided by the MS MARCO dataset.
Running this script:
python train_bi-encoder.py
"""
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import logging
from datetime import datetime
import gzip
import os
import tarfile
from collections import defaultdict
from torch.utils.data import IterableDataset

In [5]:
class TripletsDataset(IterableDataset):
    def __init__(self, model, queries, corpus, train_triplets):
        self.model = model
        self.queries = queries
        self.corpus = corpus
        self.train_triplets = train_triplets

    def __iter__(self):
        count = 0
        
        for triplet in self.train_triplets:
            qid, pos_id, neg_id = triplet
            qid = str(qid)
            pos_id = str(pos_id)
            neg_id = str(neg_id)
            query_text = self.corpus[qid]
            pos_text = self.corpus[pos_id]
            neg_text = self.corpus[neg_id]
            yield InputExample(texts=[query_text, pos_text, neg_text])

    def __len__(self):
        return len(train_triplets)

In [6]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout
train_batch_size = 64 #Increasing the train batch size improves the model performance, but requires more GPU memory

# The  model we want to fine-tune


model_name = 'distilroberta-base'
word_embedding_model = models.Transformer(model_name, max_seq_length=350)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# We construct the SentenceTransformer bi-encoder from scratch

# model = SentenceTransformer(model_name)

### Now we read the MS Marco dataset


for p in [15]:
  for n in [15]:
    p=str(p)
    n=str(n)

    #reset model
    model = SentenceTransformer(model_name)
    data_folder = '/content/drive/MyDrive/biker_data/min_5_max_'+p+'_ir_'+n+'/'
    os.makedirs(data_folder, exist_ok=True)
    import json 
    model_save_path = '/content/drive/MyDrive/SBRT_output/bi-encoder-min_5_max_'+p+'_ir_'+n+'_5k_distilroberta-base'+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    corpus = {}
    collection_filepath = os.path.join(data_folder, 'Corpus_dict.json')
    with open(collection_filepath, 'r', encoding='utf8') as fIn:
      corpus = json.load(fIn)



    Passage_dict = {}
    collection_filepath = os.path.join(data_folder, 'Passage_dict.json')
    with open(collection_filepath, 'r', encoding='utf8') as fIn:
      Passage_dict = json.load(fIn)

    evaluate_Corpus = {}
    collection_filepath = os.path.join(data_folder, 'evaluate_Corpus.json')
    with open(collection_filepath, 'r', encoding='utf8') as fIn:
      evaluate_Corpus = json.load(fIn)

    evaluate_queries = {}
    collection_filepath = os.path.join(data_folder, 'evaluate_queries.json')
    with open(collection_filepath, 'r', encoding='utf8') as fIn:
      evaluate_queries = json.load(fIn)

    evaluate_rel_doc = {}
    collection_filepath = os.path.join(data_folder, 'evaluate_rel_doc.json')
    with open(collection_filepath, 'r', encoding='utf8') as fIn:
      evaluate_rel_doc = json.load(fIn)



    def get_triplets(Passage_dict):
        Triplets= []
        for k, v in Passage_dict.items():
            for x in v[0]:
                for y in v[1]:
                    # query,same_appi,diff_api
                    Triplets.append([k,x,y])
        return Triplets

    train_triplets = get_triplets(Passage_dict)




    eval_queries = {} 
    for k in evaluate_queries:
      eval_queries[str(k)] = corpus[str(k)]
    eval_Corpus = {} 
    for k in evaluate_Corpus:
      eval_Corpus[str(k)] = corpus[str(k)]

    dev_queries =  {str(k):v for k,v in eval_queries.items()}
    dev_corpus = {str(k):v for k,v in eval_Corpus.items()}
    dev_rel_docs = {str(k):set(str(v) for v in v[0]) for k,v in evaluate_rel_doc.items()}



    ir_evaluator = evaluation.InformationRetrievalEvaluator(dev_queries, corpus, dev_rel_docs, name='distilroberta-train_eval')

    # For training the SentenceTransformer model, we need a dataset, a dataloader, and a loss used for training.
    train_dataset = TripletsDataset(model=model, queries=corpus, corpus=corpus, train_triplets=train_triplets)
    train_dataloader = DataLoader(train_dataset, shuffle=False, batch_size=train_batch_size)
    train_loss = losses.MultipleNegativesRankingLoss(model=model)


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=ir_evaluator,
              epochs=50,
              warmup_steps=1000,
              output_path=model_save_path,
              evaluation_steps=5000,
              save_best_model=True,
              use_amp=True)

2021-03-21 16:58:43 - Lock 140545143097680 acquired on /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46.lock



2021-03-21 16:58:43 - Lock 140545143097680 released on /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46.lock
2021-03-21 16:58:44 - Lock 140545133368016 acquired on /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1.lock



2021-03-21 16:58:50 - Lock 140545133368016 released on /root/.cache/huggingface/transformers/7a0115a4c463f49bc7ab011872fc4a4b81be681a0434075955d29ac3388e225b.a6127d76576e81475313180aceb31a8688f7a649b80e380d26b5d30302dc83c1.lock
2021-03-21 16:58:52 - Lock 140545104186320 acquired on /root/.cache/huggingface/transformers/23e0f7484fc8a320856b168861166b48c2976bb4e0861602422e1b0c3fe5bf61.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock



2021-03-21 16:58:53 - Lock 140545104186320 released on /root/.cache/huggingface/transformers/23e0f7484fc8a320856b168861166b48c2976bb4e0861602422e1b0c3fe5bf61.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab.lock
2021-03-21 16:58:53 - Lock 140545117227472 acquired on /root/.cache/huggingface/transformers/c7e8020011da613ff5a9175ddad64cd47238a9525db975eb50ecb965e9f7302f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock



2021-03-21 16:58:54 - Lock 140545117227472 released on /root/.cache/huggingface/transformers/c7e8020011da613ff5a9175ddad64cd47238a9525db975eb50ecb965e9f7302f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
2021-03-21 16:58:54 - Lock 140545104054800 acquired on /root/.cache/huggingface/transformers/b6a9ca6504e67903474c3fdf82ba249882406e61c2176a9d4dc9c3691c663767.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730.lock


...

2021-03-21 23:21:59 - Information Retrieval Evaluation on distilroberta-train_eval dataset in epoch 45 after 5000 steps:
2021-03-21 23:22:00 - Queries: 250
2021-03-21 23:22:00 - Corpus: 2027

2021-03-21 23:22:00 - Score-Function: cos_sim
2021-03-21 23:22:00 - Accuracy@1: 0.00%
2021-03-21 23:22:00 - Accuracy@3: 99.60%
2021-03-21 23:22:00 - Accuracy@5: 99.60%
2021-03-21 23:22:00 - Accuracy@10: 99.60%
2021-03-21 23:22:00 - Precision@1: 0.00%
2021-03-21 23:22:00 - Precision@3: 66.13%
2021-03-21 23:22:00 - Precision@5: 78.96%
2021-03-21 23:22:00 - Precision@10: 79.80%
2021-03-21 23:22:00 - Recall@1: 0.00%
2021-03-21 23:22:00 - Recall@3: 15.87%
2021-03-21 23:22:00 - Recall@5: 31.41%
2021-03-21 23:22:00 - Recall@10: 52.03%
2021-03-21 23:22:00 - MRR@10: 0.4980
2021-03-21 23:22:00 - NDCG@10: 0.7744
2021-03-21 23:22:00 - MAP@100: 0.8380
2021-03-21 23:22:00 - Score-Function: dot_score
2021-03-21 23:22:00 - Accuracy@1: 77.60%
2021-03-21 23:22:00 - Accuracy@3: 98.80%
2021-03-21 23:22:00 - Accuracy@

2021-03-21 23:30:44 - Information Retrieval Evaluation on distilroberta-train_eval dataset in epoch 46 after 5000 steps:
2021-03-21 23:30:45 - Queries: 250
2021-03-21 23:30:45 - Corpus: 2027

2021-03-21 23:30:45 - Score-Function: cos_sim
2021-03-21 23:30:45 - Accuracy@1: 0.00%
2021-03-21 23:30:45 - Accuracy@3: 99.60%
2021-03-21 23:30:45 - Accuracy@5: 99.60%
2021-03-21 23:30:45 - Accuracy@10: 99.60%
2021-03-21 23:30:45 - Precision@1: 0.00%
2021-03-21 23:30:45 - Precision@3: 66.27%
2021-03-21 23:30:45 - Precision@5: 78.96%
2021-03-21 23:30:45 - Precision@10: 79.76%
2021-03-21 23:30:45 - Recall@1: 0.00%
2021-03-21 23:30:45 - Recall@3: 15.95%
2021-03-21 23:30:45 - Recall@5: 31.41%
2021-03-21 23:30:45 - Recall@10: 51.99%
2021-03-21 23:30:45 - MRR@10: 0.4980
2021-03-21 23:30:45 - NDCG@10: 0.7744
2021-03-21 23:30:45 - MAP@100: 0.8385
2021-03-21 23:30:45 - Score-Function: dot_score
2021-03-21 23:30:45 - Accuracy@1: 76.80%
2021-03-21 23:30:45 - Accuracy@3: 98.00%
2021-03-21 23:30:45 - Accuracy@

2021-03-21 23:39:29 - Information Retrieval Evaluation on distilroberta-train_eval dataset in epoch 47 after 5000 steps:
2021-03-21 23:39:30 - Queries: 250
2021-03-21 23:39:30 - Corpus: 2027

2021-03-21 23:39:30 - Score-Function: cos_sim
2021-03-21 23:39:30 - Accuracy@1: 0.00%
2021-03-21 23:39:30 - Accuracy@3: 99.60%
2021-03-21 23:39:30 - Accuracy@5: 99.60%
2021-03-21 23:39:30 - Accuracy@10: 99.60%
2021-03-21 23:39:30 - Precision@1: 0.00%
2021-03-21 23:39:30 - Precision@3: 66.13%
2021-03-21 23:39:30 - Precision@5: 78.96%
2021-03-21 23:39:30 - Precision@10: 79.76%
2021-03-21 23:39:30 - Recall@1: 0.00%
2021-03-21 23:39:30 - Recall@3: 15.87%
2021-03-21 23:39:30 - Recall@5: 31.41%
2021-03-21 23:39:30 - Recall@10: 52.00%
2021-03-21 23:39:30 - MRR@10: 0.4980
2021-03-21 23:39:30 - NDCG@10: 0.7742
2021-03-21 23:39:30 - MAP@100: 0.8381
2021-03-21 23:39:30 - Score-Function: dot_score
2021-03-21 23:39:30 - Accuracy@1: 78.00%
2021-03-21 23:39:30 - Accuracy@3: 98.00%
2021-03-21 23:39:30 - Accuracy@

2021-03-21 23:48:16 - Information Retrieval Evaluation on distilroberta-train_eval dataset in epoch 48 after 5000 steps:
2021-03-21 23:48:17 - Queries: 250
2021-03-21 23:48:17 - Corpus: 2027

2021-03-21 23:48:17 - Score-Function: cos_sim
2021-03-21 23:48:17 - Accuracy@1: 0.00%
2021-03-21 23:48:17 - Accuracy@3: 99.60%
2021-03-21 23:48:17 - Accuracy@5: 99.60%
2021-03-21 23:48:17 - Accuracy@10: 99.60%
2021-03-21 23:48:17 - Precision@1: 0.00%
2021-03-21 23:48:17 - Precision@3: 66.13%
2021-03-21 23:48:17 - Precision@5: 78.96%
2021-03-21 23:48:17 - Precision@10: 79.76%
2021-03-21 23:48:17 - Recall@1: 0.00%
2021-03-21 23:48:17 - Recall@3: 15.87%
2021-03-21 23:48:17 - Recall@5: 31.41%
2021-03-21 23:48:17 - Recall@10: 52.00%
2021-03-21 23:48:17 - MRR@10: 0.4980
2021-03-21 23:48:17 - NDCG@10: 0.7742
2021-03-21 23:48:17 - MAP@100: 0.8379
2021-03-21 23:48:17 - Score-Function: dot_score
2021-03-21 23:48:17 - Accuracy@1: 78.40%
2021-03-21 23:48:17 - Accuracy@3: 98.00%
2021-03-21 23:48:17 - Accuracy@

2021-03-21 23:57:02 - Information Retrieval Evaluation on distilroberta-train_eval dataset in epoch 49 after 5000 steps:
2021-03-21 23:57:02 - Queries: 250
2021-03-21 23:57:02 - Corpus: 2027

2021-03-21 23:57:02 - Score-Function: cos_sim
2021-03-21 23:57:02 - Accuracy@1: 0.00%
2021-03-21 23:57:02 - Accuracy@3: 99.60%
2021-03-21 23:57:02 - Accuracy@5: 99.60%
2021-03-21 23:57:02 - Accuracy@10: 99.60%
2021-03-21 23:57:02 - Precision@1: 0.00%
2021-03-21 23:57:02 - Precision@3: 66.13%
2021-03-21 23:57:02 - Precision@5: 78.96%
2021-03-21 23:57:02 - Precision@10: 79.76%
2021-03-21 23:57:02 - Recall@1: 0.00%
2021-03-21 23:57:02 - Recall@3: 15.87%
2021-03-21 23:57:02 - Recall@5: 31.41%
2021-03-21 23:57:02 - Recall@10: 52.00%
2021-03-21 23:57:02 - MRR@10: 0.4973
2021-03-21 23:57:02 - NDCG@10: 0.7741
2021-03-21 23:57:02 - MAP@100: 0.8378
2021-03-21 23:57:02 - Score-Function: dot_score
2021-03-21 23:57:02 - Accuracy@1: 78.40%
2021-03-21 23:57:02 - Accuracy@3: 98.00%
2021-03-21 23:57:02 - Accuracy@

In [ ]:
model.evaluate(evaluator=ir_evaluator)

2021-03-03 04:42:32 - Information Retrieval Evaluation on distilroberta-train_eval dataset:
2021-03-03 04:42:42 - Accuracy@1: 0.00%
2021-03-03 04:42:42 - Accuracy@3: 58.30%
2021-03-03 04:42:42 - Accuracy@5: 68.80%
2021-03-03 04:42:42 - Accuracy@10: 80.70%
2021-03-03 04:42:42 - Precision@1: 0.00%
2021-03-03 04:42:42 - Precision@3: 30.07%
2021-03-03 04:42:42 - Precision@5: 34.60%
2021-03-03 04:42:42 - Precision@10: 37.60%
2021-03-03 04:42:42 - Recall@1: 0.00%
2021-03-03 04:42:42 - Recall@3: 2.00%
2021-03-03 04:42:42 - Recall@5: 3.85%
2021-03-03 04:42:42 - Recall@10: 7.52%
2021-03-03 04:42:42 - MRR@10: 0.3102
2021-03-03 04:42:42 - NDCG@10: 0.3341
2021-03-03 04:42:42 - MAP@100: 0.2899
2021-03-03 04:42:42 - Queries: 1000
2021-03-03 04:42:42 - Corpus: 21481



0.28987622595921436

In [ ]:
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('msmarco-distilbert-base-v2')

dev_q = list(dev_queries.items())[:10]
# dev_r = list(dev_rel_docs.items())[:10]

for (k,v) in dev_q:
  query = v
  passage_id = list(dev_rel_docs[k])[0]
  passage = dev_corpus[passage_id]
  print(query)
  print(passage_id)
  print(passage)
  query_embedding = model.encode(query)
  passage_embedding = model.encode(passage)

  print("Similarity:", util.pytorch_cos_sim(query_embedding, passage_embedding))